# The FRI Protocol

**Module 10** | 10-snarks-starks

*Polynomial commitment, folding, Reed-Solomon*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **the fri protocol**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Groth16 Overview](10d-groth16-overview.ipynb).
- Concepts and notation introduced in the previous notebook.

## Reed-Solomon Codes

In [ ]:
# A low-degree polynomial evaluated on a domain forms a codeword
# FRI proves that a function is close to a low-degree polynomial
# TODO: demonstrate Reed-Solomon encoding

## The Folding Technique

In [ ]:
# Split f(x) into even and odd parts, combine with random challenge
# f(x) = f_even(x^2) + x * f_odd(x^2)
# New polynomial has half the degree
# TODO: implement one round of FRI folding

## FRI as Polynomial Commitment

In [ ]:
# Repeated folding until constant polynomial
# Verifier checks consistency with Merkle proofs
# TODO: outline the full FRI protocol

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **the fri protocol**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [STARKs vs SNARKs](10f-starks-vs-snarks.ipynb)